<a href="https://colab.research.google.com/github/Chiranjeevi2001/coberx-gemma2b/blob/main/Gemma2bFinetuningCOBREX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install peft datasets transformers trl accelerate bitsandbytes -q
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModelForCausalLM
from trl import SFTTrainer
from datasets import Dataset
from huggingface_hub import notebook_login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
save_path = "/content/dataset_cbl.json"
dataset_train_name = 'EN-train'
# dataset_val_name = 'EN-val'
# file_name_train_chatml = f"{dataset_train_name}_chatml.json"
# file_name_val_chatml = f"{dataset_val_name}_chatml.json"

with open(save_path, 'r', encoding='latin-1') as f:
  dataset_train = Dataset.from_dict(json.load(f))

# we will keep this as optional
# with open(save_path + file_name_val_chatml, 'r') as f:
#   dataset_val = Dataset.from_dict(json.load(f))

In [3]:
notebook_login()

In [29]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
tokenizer.padding_side = 'right'
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="auto",
    quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
training_arguments = TrainingArguments(
    output_dir = "/content/output_adapters/",
    # evaluation_strategy="steps",
    logging_strategy="steps",
    lr_scheduler_type="constant",
    logging_steps=20,
    # eval_steps=20,
    save_steps=20,
    per_device_train_batch_size=2,
    # per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    # eval_accumulation_steps=16,
    num_train_epochs=1,
    fp16=True,
    group_by_length = True,
    optim="paged_adamw_32bit",
    max_steps = 100
)
peft_config = LoraConfig(
    lora_alpha = 16,
    lora_dropout=0.1,
    r=16,
    task_type='CAUSAL_LM'
)
trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    # eval_dataset=dataset_val,
    peft_config=peft_config,
    neftune_noise_alpha=5,
    max_seq_length=500,
    args = training_arguments
)

Map:   0%|          | 0/393 [00:00<?, ? examples/s]

In [36]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 976.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 935.06 MiB is free. Process 2236 has 13.83 GiB memory in use. Of the allocated memory 13.58 GiB is allocated by PyTorch, and 123.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [11]:
trainer.model.save_pretrained("/content/output_adapters")

In [12]:
finetuned_model = PeftModelForCausalLM.from_pretrained(model=model, model_id="/content/output_adapters")

In [21]:
messages = [
    {
        'role':'user',
        'content': "IDENTIFICATIONDIVISION.\nPROGRAM-ID.PROD2V1.\nENVIRONMENTDIVISION.\nCONFIGURATIONSECTION.\nSPECIAL-NAMES.\nINPUT-OUTPUTSECTION.\nFILE-CONTROL.\nSELECTRENTALASSIGNTO'RENTACAR-IN.txt'\nORGANISATIONISLINESEQUENTIAL.\nSELECTRENTAL-OUTASSIGNTO'RENTACAR-OUT.txt'.\nDATADIVISION.\nFILESECTION.\nFDRENTAL.\n01RENTAL-FILE.\n02CLIENT_NAMEPICA(20).\n02RENTAL-TYPE.\n03NAME_INITIALPICA(1).\n03CAR_TYPEPIC9(1).\n03KILOMETERSPIC9(5).\n03NUM_DAYSPIC9(3).\nFDRENTAL-OUT.\n01RENTAL-FILE-OUT.\n02CLIENT_NAME_OUTPICA(20).\n02FILLERPICX(1)VALUESPACE.\n02NAME_INITIAL_OUTPICA(1).\n02FILLERPICX(1)VALUESPACE.\n02CAR_BRANDPICA(10).\n02FILLERPICX(1)VALUESPACE.\n02KILOMETERS_OUTPICzzz99.\n02FILLERPICX(1)VALUESPACE.\n02NUM_DAYS_OUTPICzz9.\n02FILLERPICX(1)VALUESPACE.\n02PAYMENTPICzz99.99.\nWORKING-STORAGESECTION.\n77END-LOOPPICA(3)VALUESPACES.\n77KILOMETERS_PAYMENTPIC9(4)V99.\n77RENTAL_DAYS_TOTALPIC9(4).\nPROCEDUREDIVISION.\nMAIN-PROCEDURE.\nOPENINPUTRENTAL\nOUTPUTRENTAL-OUT.\nREADRENTAL\nATENDMOVE\"YYY\"TOEND-LOOP\nEND-READ.\nPERFORMCALCULATIONSUNTILEND-LOOP=\"YYY\".\nCLOSERENTAL\nRENTAL-OUT.\nSTOPRUN.\nCALCULATIONS.\nMOVECLIENT_NAMETOCLIENT_NAME_OUT.\nMOVENAME_INITIALTONAME_INITIAL_OUT.\nMOVEKILOMETERSTOKILOMETERS_OUT.\nMOVENUM_DAYSTONUM_DAYS_OUT.\nIFKILOMETERSISGREATERTHAN75OREQUALTO75\nCOMPUTEKILOMETERS=KILOMETERS-75.\nEVALUATECAR_TYPE\nWHEN1MOVE\"VOLKSWAGEN\"TOCAR_BRAND\nCOMPUTEKILOMETERS_PAYMENT=KILOMETERS*0.5\nCOMPUTERENTAL_DAYS_TOTAL=10*NUM_DAYS\nCOMPUTEPAYMENT=KILOMETERS_PAYMENT+RENTAL_DAYS_TOTAL\nWHEN2MOVE\"TOYOTA\"TOCAR_BRAND\nCOMPUTEKILOMETERS_PAYMENT=KILOMETERS*0.55\nCOMPUTERENTAL_DAYS_TOTAL=12.5*NUM_DAYS\nCOMPUTEPAYMENT=KILOMETERS_PAYMENT+RENTAL_DAYS_TOTAL\nWHEN3MOVE\"MERCEDES\"TOCAR_BRAND\nCOMPUTEKILOMETERS_PAYMENT=KILOMETERS*0.65\nCOMPUTERENTAL_DAYS_TOTAL=16*NUM_DAYS\nCOMPUTEPAYMENT=KILOMETERS_PAYMENT+RENTAL_DAYS_TOTAL\nEND-EVALUATE.\nWRITERENTAL-FILE-OUT\nAFTERSADVANCING1LINE.\nDISPLAYRENTAL-FILE-OUT.\nREADRENTAL\nATENDMOVE\"YYY\"TOEND-LOOP\nEND-READ.\nLEGACY.\nSTOPRUN.\nENDPROGRAMPROD2V1.\n"
    }
]

In [22]:
input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_tensors="pt").to("cuda")

# print(input_ids)
outputs_finetuned = finetuned_model.generate(input_ids=input_ids, max_new_tokens=1024, do_sample=False)
# outputs = model_2.generate(input_ids=input_ids, max_new_tokens=1024, do_sample=False)

print("finetuned: " + tokenizer.decode(outputs_finetuned[0]).split('<start_of_turn>model\n')[-1])
# print("normal   : " + tokenizer.decode(outputs[0]).split('<start_of_turn>model\n')[-1])

finetuned: This is a well-written and documented program specification for a program called "PROD2V1". The program is designed to manage the rental of vehicles, and it includes the following sections:

* **Identification Division**
    * Identifies the program and its purpose.
    * Defines the program identifier and environment division.
    * Defines the configuration section.
    * Defines the input-output section.
    * Defines the data division.
    * Defines the file-control section.
* **Program Division**
    * Defines the main-procedure.
    * Defines the procedure division.
    * Defines the main-procedure.
* **Main-Procedure**
    * Opens input file "RENTACAR-IN.txt".
    * Reads data from file "RENTACAR-IN.txt".
    * Creates a file "RENTACAR-OUT.txt".
    * Writes data to file "RENTACAR-OUT.txt".
    * Closes input file "RENTACAR-IN.txt".
    * Closes output file "RENTACAR-OUT.txt".
    * Performs calculations and writes results to file "RENTACAR-OUT.txt".
    * Stops the p